<a href="https://colab.research.google.com/github/djolobolonjez/PSIML8-Qualification-tasks/blob/master/Task%201/PSIML8_Rock_my_World.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""from zipfile import ZipFile
file_name = "/rockmyworld-public.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')"""

Importing libraries

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import math
from pathlib import Path

Input and global variables

In [ ]:
path = input()
filenum = 0
citymap = {}
countrymap = {}
visitors = {}
venues = {}
indie = {}
bands = {}
tours = {}
count = 0
concerts = 0
df_list = []

Extracting data

In [ ]:
def info_extract(dir, path, file):
    d = Path(dir)
    p = Path(path)
    d_list = list(d.parts)
    p_list = list(p.parts)

    global count, concerts

    venue = os.path.splitext(file)[0]
    year = month = day = city = country = ""
    for i in d_list:
        p_list.remove(i)
    if p_list[0].isnumeric():
        year = p_list[0]
        month = p_list[1]
        day = p_list[2]
        city = p_list[-1]
        country = p_list[-2]
    elif p_list[0].find("_") != -1 and p_list[0][0].isnumeric():
        splitted = p_list[0].split("_")
        year = splitted[0]
        month = splitted[1]
        day = splitted[2]
        city = p_list[-1]
        country = p_list[-2]
    else:
        splitted = p_list[-1].split("_")
        year = splitted[0]
        month = splitted[1]
        day = splitted[2]
        city = p_list[1]
        country = p_list[0]

    month = month.strip("0")
    day = day.strip("0")

    date = str(day) + "_" + str(month) + "_" + str(year)
    country = country.replace("-", "_")
    city = city.replace("-", "_")
    if country[:4].lower() == 'the_' or country[:4].lower() == 'the-':
        country = country[4:]

    os.chdir(path)
    with open(file, 'r') as f:
        x = f.readlines()

        if not len(x):
            return
        elif x[-1] == ']':
            data = json.load(open(file, "r"))
            df = pd.DataFrame(data)
        else:
            data = [json.loads(line) for line in open(file, 'r')]
            df = pd.DataFrame(data)
    if not df.empty:
        if 'band_name' in df.columns:
            #df.dropna(subset=['band_name'], inplace=True)
            valid = df['band_name'].count()
            #concerts += valid
        else:
            return

        if 'is_indie' not in df.columns:
            df['is_indie'] = False
        else:
            df['is_indie'].fillna(False, inplace=True)

        if any(df['is_indie']):
          for band in df['band_name']:
            if not pd.isna(band):
              if venue not in venues:
                venues[venue] = [[band], date]
              else:
                venues[venue][0].append(band)
        else:
          if venue in venues:
            if date == venues[venue][-1]:
              for band in df['band_name']:
                if not pd.isna(band):
                  venues[venue][0].append(band)
          

        if 'attendance' in df.columns:
            count += df['attendance'].mean()

        if country not in countrymap:
            countrymap[country] = valid
        else:
            countrymap[country] += valid
        if city not in citymap:
            citymap[city] = valid
        else:
            citymap[city] += valid

        df_list.append([df, city])

Counting number of JSON files

In [ ]:
def file_count(dir):
  global filenum
  for roots, dirs, files in os.walk(dir):
    for file in files:
      filenum += 1
      info_extract(dir, roots, file)

Filling some missing values

In [ ]:
def fill_attendance():
  average = count/filenum
  for elem in df_list:
    df = elem[0]
    if 'attendance' not in df.columns:
      df['attendance'] = average
    else:
      mean_value = df["attendance"].mean(skipna = True)
      if not math.isnan(mean_value):
        df['attendance'].fillna(value = mean_value, inplace = True)
      else:
        df['attendance'].fillna(value = average, inplace = True)

Part 4 of the task

In [ ]:
def most_indie():
  average = count/filenum
  for elem in df_list:
    df = elem[0]
    city = elem[-1]
    sum = df.loc[df['is_indie'] == True, 'attendance'].sum()
    if city not in indie:
      indie[city] = sum
    else:
      indie[city] += sum
    

Part 5 of the task, needs some complexity improving

In [ ]:
def top_bands():
  for elem in df_list:
    df = elem[0]
    for band in df['band_name']:
      if band in bandset:
        num = df.loc[df['band_name'] == band, 'attendance'].iloc[0]
        if band not in bands and band not in tours:
          bands[band] = num
          tours[band] = 1
        else:
          bands[band] += num
          tours[band] += 1

  for band in bands:
    bands[band] /= tours[band]
  
  return dict(sorted(bands.items(), key = lambda item: item[1], reverse = True))
          

Main function

In [ ]:
file_count(path)
print(filenum)
print(len(countrymap))
max = 0
city = ""
for key in citymap:
  if citymap[key] > max:
    max = citymap[key]
    city = key
  elif citymap[key] == max:
    if key < city:
      max = citymap[key]
      city = key

print(city)
fill_attendance()
most_indie()
indie = dict(sorted(indie.items(), key=lambda item: item[1], reverse = True))
most = list(indie.keys())[:3]
print(*most, sep = ",")

bnds = []
for key in venues:
  bnds += [elem for elem in venues[key][0]]

bandset = set(bnds)
best_bands = list(top_bands())[:3]
print(*best_bands, sep = ",")